In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.constraints import max_norm, min_max_norm
from tensorflow.keras.optimizers import SGD, Adam, Nadam

In [2]:
data = pd.read_csv("../input/preprocess-completed/preprocess_completed.csv")
X = data.loc[:, ["venue", "innings", "batting_team","n_wickets"]]
y = data.iloc[:, 6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=22)

In [3]:
def create_model_batting(dropout_rate=0.22, activation="tanh", init_mode='uniform', weight_constraint=3.4, learn_rate=0.000068, momentum=0.28):
    model = Sequential()
    model.add(Dense(12, activation = activation, input_dim = 4, kernel_initializer=init_mode, kernel_constraint=min_max_norm(0.15, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(10, activation = activation, kernel_initializer=init_mode,kernel_constraint=min_max_norm(0.15, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(10, activation = activation, kernel_initializer=init_mode,kernel_constraint=min_max_norm(0.15, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(8, activation = activation, kernel_initializer=init_mode,kernel_constraint=min_max_norm(0.1, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(6, activation = activation, kernel_initializer=init_mode,kernel_constraint=min_max_norm(0.1, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(6, activation = activation, kernel_initializer=init_mode,kernel_constraint=min_max_norm(0.05, weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", optimizer = SGD(learning_rate=learn_rate, momentum=momentum), metrics=['mean_squared_error'])
    return model

In [4]:
batting_model = create_model_batting()
es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=3)

In [5]:
history = batting_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4000, batch_size=18, verbose=0, callbacks=[es])

Epoch 00017: early stopping


In [6]:
print(history.history['loss'][-1], history.history['val_loss'][-1])

132.983154296875 111.06031799316406


In [7]:
save_model(batting_model, "batting_model")